In [1]:
# run once (may have to restart notebook)
# !pip install tensorflow-gpu==1.15 --user
# !pip install gpt-2-simple --user

In [1]:
import pandas as pd
import numpy as np
import json
import gpt_2_simple as gpt2
import os
import requests

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
# check to make sure gpu is recognized for significantly faster training

# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [3]:
model_name = "124M"
if not os.path.isdir(os.path.join("models", model_name)):
    print(f"Downloading {model_name} model...")
    gpt2.download_gpt2(model_name=model_name)

In [4]:
dir_path = "../data/"

dem_file_name = "democrats_result.txt"
dem_sample_name = "democrats_sample.txt"
dem_model_name = 'dem'

rep_file_name = 'republican_result.txt'
rep_sample_name = 'republican_sample.txt'
rep_model_name = 'rep'

both_file_name = 'both_result.txt'
both_sample_name = 'both_sample.txt'
both_model_name = 'both'

In [5]:
# reading and writing sample files for each side

# with open(dir_path+dem_file_name,'r') as demf:
#     dem_data = demf.readlines(500000)
# with open(dir_path+dem_sample_name,'w+') as dem_write:
#     dem_write.writelines(dem_data)

# with open(dir_path+rep_file_name,'r') as repf:
#     rep_data = repf.readlines(500000)
# with open(dir_path+rep_sample_name,'w+') as rep_write:
#     rep_write.writelines(rep_data)

# dem_data.extend(rep_data)
# both_data = dem_data


# with open(dir_path+both_file_name,'r') as bothf:
#     both_data = bothf.readlines(100000)
# with open(dir_path+both_sample_name,'w+') as both_write:
#     both_write.writelines(both_data)

In [6]:
# text file to train model on
train_fp = dir_path + dem_sample_name
train_name = dem_model_name

In [7]:
# this cell takes the longest. Can only be run once without restarting the notebook
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              train_fp,
              model_name=model_name,
              steps=100, # steps is max number of training steps
              restore_from='fresh', # makes sure model doesnt resume from previous trained model
              print_every=20, # only prints every 20 training steps,
              run_name=train_name # model name, so we can load different models locally
             )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:01<00:00,  1.16s/it]


dataset has 115022 tokens
Training...
[20 | 33.96] loss=3.67 avg=3.67
[40 | 59.58] loss=2.98 avg=3.32
[60 | 85.27] loss=3.16 avg=3.27
[80 | 110.99] loss=3.02 avg=3.20
[100 | 136.74] loss=2.55 avg=3.07
Saving checkpoint/dem/model-100


In [9]:
# prompt to generate response to, going to be a post/comment from the political discussion subreddits

pre = "After witnessing the heinous amount of police brutality in the US against POC, \
my partner and I have created some protest posters in response. We do not care about \
credit, just getting the message out there, so feel free to distribute however way you \
want, or make better versions yourself if you think it can be improved!"

In [10]:
gpt2.generate(sess, 
              temperature=.8, # uniqueness of the output (usually ranges from .5 to 2)
              prefix=pre, # prompt
              nsamples=5, # number of generated responses 
              length=400 # number of words (including prompt) per response
             )

After witnessing the heinous amount of police brutality in the US against POC, my partner and I have created some protest posters in response. We do not care about credit, just getting the message out there, so feel free to distribute however way you want, or make better versions yourself if you think it can be improved!


Still, we are not communists, we are taking a stand for justice and the rule of law.


So please spread the word and feel free to share your thoughts. We will do our best to keep our voice heard.


And if anyone does not feel like now is the time to go further, we are here to support justice and for the rule of law. Thanks for your leadership and understanding. I'm afraid we have a long way to go

We will use our power to take back our country. We will not stand idly by when the corrupt, powerful government is us.

We will take back our country from those that want to destroy it. But we will not stand idly by when it comes to the violent execution of our fellow Ameri